In [1]:
import pandas as pd
import numpy as np
import re
import math
from cleantext import clean
import matplotlib.pyplot as plt
import nltk
from nltk.stem.snowball import SnowballStemmer
from collections import Counter
import csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn import svm


nltk.download('punkt')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 150)

C:\Users\astri\AppData\Local\Temp\ipykernel_13024\1475292071.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\astri\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from sklearn.model_selection import train_test_split
#remove unknowns
relevant = ['fake','reliable']
news_data =pd.read_csv("small_datasetprocessed.csv")

#sorting data
data = news_data[news_data["simple_type"].isin(relevant) == True] #fjerne ikke-relevante rækker, simple type tracker definitionen af fake news 

print("row count after removal:", len(data))
print(list(data))

X = data.drop(["type"],axis=1).drop(["simple_type"],axis=1) #ikke sikker på de skal være i en liste
#tjek lige om der er andre mærkelige kolonner vi har lavet i det?
y = data["simple_type"] #eller type?

X_train, X_val, y_train, y_val= train_test_split(X,y,test_size=0.2,random_state=0,shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_val,y_val, test_size=0.5,random_state=0,shuffle=True)
 
#tjek om de stadig passer sammen
#spørg til:
#1: I opgaven virker det som om at vi først skal splitte data før vi deler i fake og reliable - det giver ikke mening umiddelbart


row count after removal: 195
['Unnamed: 0.1', 'Unnamed: 0', 'id', 'domain', 'type', 'url', 'content', 'scraped_at', 'inserted_at', 'updated_at', 'title', 'authors', 'keywords', 'meta_keywords', 'meta_description', 'tags', 'summary', 'simple_type', 'mean_word_length', 'unique_words_ratio', 'content_no_stopwords', 'content_stemmed', 'length', 'numbers']


In [52]:
#baseline model 1: single-feature logistic regression, length of article

def bin_target(x):
    if x=="reliable":
        return 0
    if x=="fake":
        return 1

#article_length_train = X_train["content_stemmed"].apply(lambda x: len(x)) #get feature with length of article (overvej om det er stemmed man skal gøre det på?)
#article_length_test = X_test["content_stemmed"].apply(lambda x: len(x)) #get feature with length of article (overvej om det er stemmed man skal gøre det på?)

bin_y_train = y_train.apply(lambda x:bin_target(x)) #get binary target
bin_y_test = y_test.apply(lambda x:bin_target(x)) #get binary target

#model = LogisticRegression()
#reg = model.fit(pd.DataFrame(article_length_train),bin_y_train)

#y_pred = model.predict(pd.DataFrame(article_length_test)) #er det test eller validation?
#acc = accuracy_score(y_pred,bin_y_test)

#print(acc)


1.0


In [47]:
def gridSearch(solver,penalties,data,target):
    model = LogisticRegression()
    parameters = {'solver' : solver,'penalty': penalties, 'C' : [0.001, 0.01, 0.1, 1.0, 10]}
    clf = GridSearchCV(model, parameters, scoring='accuracy', cv=5)
    clf.fit(data,target)
    return clf

#l2, 1.0, lbfgs
#uniquewords1 = gridSearch(["liblinear"], ["l2","l2"], pd.DataFrame(X_train["unique_words_ratio"]), bin_y_train)
#uniquewords2 = gridSearch(["newton-cg", "sag","lbfgs"], ["l2","None"],pd.DataFrame(X_train["unique_words_ratio"]), bin_y_train)
#uniquewords3 = gridSearch(["saga"], ["elastic-net","l1","l2","None"], pd.DataFrame(X_train["unique_words_ratio"]), bin_y_train)

wordlen1 = gridSearch(["liblinear"], ["l2","l2"], pd.DataFrame(X_train["mean_word_length"]), bin_y_train)
wordlen2 = gridSearch(["newton-cg", "sag","lbfgs"], ["l2","None"],pd.DataFrame(X_train["mean_word_length"]), bin_y_train)
wordlen3 = gridSearch(["saga"], ["elastic-net","l1","l2","None"], pd.DataFrame(X_train["mean_word_length"]), bin_y_train)

#articlelen1 = gridSearch(["liblinear"], ["l2","l2"], pd.DataFrame(article_length_train), bin_y_train)
#articlelen2 = gridSearch(["newton-cg", "sag","lbfgs"], ["l2","None"],pd.DataFrame(article_length_train), bin_y_train)
#articlelen3 = gridSearch(["saga"], ["elastic-net","l1","l2","None"], pd.DataFrame(article_length_train), bin_y_train)

baselines = [uniquewords1,uniquewords2,uniquewords3, wordlen1,wordlen2,wordlen3,articlelen1,articlelen2,articlelen3]
for baseline in baselines:
    bestscore = 0
    bestmodel = uniquewords1
    if baseline.best_score_>bestscore:
        bestscore = baseline.best_score_
        bestmodel = baseline.best_params_

print('Baseline model is logistic regression with parameters ',bestmodel)
print("with accuracy:", bestscore)

#kan måske være fordi der er for få reliable artikler?

c:\Users\astri\miniconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\astri\miniconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\astri\miniconda3\Lib\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
75 fits failed out of a total of 150.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
75 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\astri\miniconda3\Lib\site-packages\sklearn\model_selection\_validation.

Baseline model is logistic regression with parameters  {'C': 0.001, 'penalty': 'l1', 'solver': 'saga'}
with accuracy: 0.9872983870967742


c:\Users\astri\miniconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\astri\miniconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\astri\miniconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\astri\miniconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\astri\miniconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\astri\miniconda3\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max

In [48]:
print(articlelen2.best_score_)

0.9872983870967742


In [49]:
print('Baseline model is logistic regression with parameters ',bestmodel)
print("with accuracy:", bestscore)
#which is weird?
   

Baseline model is logistic regression with parameters  {'C': 0.001, 'penalty': 'l1', 'solver': 'saga'}
with accuracy: 0.9872983870967742


In [50]:
#add reliable data
addata = pd.read_csv("reliables")

X_rel = data.drop(["type"],axis=1).drop(["simple_type"],axis=1)
y_rel = data["simple_type"]

X_train_rel, X_val_rel, y_train_rel, y_val_rel= train_test_split(X_rel,y_rel,test_size=0.2,random_state=0,shuffle=True)
X_val_rel, X_test_rel, y_val_rel, y_test_rel = train_test_split(X_val_rel,y_val_rel, test_size=0.5,random_state=0,shuffle=True)

X_train = pd.concat([X_train,X_train_rel])
y_train = pd.concat([y_train,y_train_rel])



1.0
